In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Q1(d)


# Define the range of theta values
theta = np.linspace(0, 2*np.pi, 1000)

# Define the range of alpha0 values
alpha0_values1 = np.arange(-1.8, -1.0, 0.1)
alpha0_values2 = np.arange(-1.0, 0.0, 0.1)

# Figure 1: Plot the boundaries of the RAS for alpha0 = -1.8, -1.7, ..., -1.1
plt.figure(1)
for alpha0 in alpha0_values1:
    alpha1 = -1 - alpha0
    beta0 = (2/3 + alpha0/3) * 2
    beta_m1 = (4/3 + alpha0/6) /2
    beta1 = 2 + alpha0 - beta_m1 - beta0
    
    rho = np.exp(1j*theta)**2 + alpha0*np.exp(1j*theta) + alpha1
    sigma = beta_m1*np.exp(1j*theta)**2 + beta0*np.exp(1j*theta) + beta1
    
    h_lambda = rho / sigma
    
    plt.plot(h_lambda.real, h_lambda.imag, label=f'$\\alpha_0 = {alpha0:.1f}$')
plt.xlabel('Real')
plt.ylabel('Imaginary')
plt.title('Boundaries of RAS for $\\alpha_0 = -1.8, -1.7, ..., -1.1$')
plt.legend(loc='best')

# Figure 2: Plot the boundaries of the RAS for alpha0 = -1.0, -0.9, ..., -0.1
plt.figure(2)
for alpha0 in alpha0_values2:
    alpha1 = -1 - alpha0
    beta0 = 4/3 + alpha0 * 2/3
    beta_m1 = (2 + alpha0/2 - beta0) / 2
    beta1 = 2 + alpha0 - beta_m1 - beta0
    
    rho = np.exp(1j*theta)**2 + alpha0*np.exp(1j*theta) + alpha1
    sigma = beta_m1*np.exp(1j*theta)**2 + beta0*np.exp(1j*theta) + beta1
    
    h_lambda = rho / sigma
    
    plt.plot(h_lambda.real, h_lambda.imag, label=f'$\\alpha_0 = {alpha0:.1f}$')
plt.xlabel('Real')
plt.ylabel('Imaginary')
plt.title('Boundaries of RAS for $\\alpha_0 = -1.0, -0.9, ..., -0.1$')
plt.legend(loc='best')

plt.show()

In [ ]:
#q1 (e)
def check_root_condition(alpha0, z):
    alpha1 = -1 - alpha0
    beta0 = (2/3 + alpha0/3) * 2
    beta_m1 = (4/3 + alpha0/6) /2
    beta1 = 2 + alpha0 - beta_m1 - beta0
    
    rho_z = z**2 + alpha0*z + alpha1
    sigma_z = beta_m1*z**2 + beta0*z + beta1
    
    return abs(rho_z) <= abs(sigma_z)


# Check the root condition for each contour in Figure 1
for alpha0 in alpha0_values1:
    z = complex(-5, 0)  # Choose a point inside each contour
    if check_root_condition(alpha0, z):
        print(f"For z = {z}, α0 = {alpha0:.1f}, the RAS lies outside the contour.")
    else:
        print(f"For z = {z}, α0 = {alpha0:.1f}, the RAS lies inside the contour.")

# Check the root condition for each contour in Figure 2
print("\nFigure 2:")
for alpha0 in alpha0_values2:
    z = complex(-1e-2, 0)  # Choose a point inside each contour
    if check_root_condition(alpha0, z):
        print(f"For z = {z}, α0 = {alpha0:.1f}, the RAS lies outside the contour.")
    else:
        print(f"For z = {z}, α0 = {alpha0:.1f}, the RAS lies inside the contour.")

In [ ]:
#q3 (b)

import numpy as np
import matplotlib.pyplot as plt

def f(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        x * r**-3/2,
        y * r**-3/2,
        u,
        v
    ])

def Df(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        [1/r**3 - 3*x**2/r**5, -3*x*y/r**5, 0, 0],
        [-3*x*y/r**5, 1/r**3 - 3*y**2/r**5, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def implicit_midpoint(z0, h, n_steps):
    z = np.zeros((n_steps+1, 4))
    z[0] = z0
    
    for i in range(n_steps):
        k = f(z[i]) + 0.5 * h * np.dot(Df(z[i]), f(z[i]))
        for _ in range(10):
            k = f(z[i] + 0.5 * h * k) + 0.5 * h * np.dot(Df(z[i] + 0.5 * h * k), k)
        z[i+1] = z[i] + h * k
    
    return z

# Parameters
u0, v0, x0, y0 = 0, 0.5, 2, 0
z0 = np.array([u0, v0, x0, y0])
T = 9.673596609249161
n_revs = 10

# Integrate with different time steps
steps_per_period = [100, 1000, 10000]

for spp in steps_per_period:
    n_steps = n_revs * spp
    h = T / spp
    z = implicit_midpoint(z0, h, n_steps)
    
    # Plot x and y components
    plt.figure()
    plt.plot(z[:, 2], z[:, 3])
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Trajectory (steps per period = {spp})')
    plt.axis('equal')
    
    # Plot Hamiltonian versus time
    H = 0.5 * (z[:, 0]**2 + z[:, 1]**2) - 1 / np.sqrt(z[:, 2]**2 + z[:, 3]**2)
    t = np.linspace(0, n_revs*T, n_steps+1)
    plt.figure()
    plt.plot(t, H)
    plt.xlabel('Time')
    plt.ylabel('Hamiltonian')
    plt.title(f'Hamiltonian vs Time (steps per period = {spp})')

plt.show()

In [ ]:
import sympy as sym

def Jacobian(v_str, f_list):
    vars = sym.symbols(v_str)
    f = sym.sympify(f_list)
    J = sym.zeros(len(f),len(vars))
    for i, fi in enumerate(f):
        for j, s in enumerate(vars):
            J[i,j] = sym.diff(fi, s)
    return J

Jacobian('u v x y', ['x*(x^2+y^2)**(-3/2)','y*(x^2+y^2)**(-3/2)', 'u', 'v'])

In [41]:
import numpy as np

def f(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        x / r**3,
        y / r**3,
        u,
        v
    ])

def Df(z):
    u, v, x, y = z
    r2 = x**2 + y**2
    r3 = r2 ** (3/2)
    r5 = r2 ** (5/2)
    return np.array([
        [0, 0, -3*x**2/r5 + 1/r3, -3*x*y/r5],
        [0, 0, -3*x*y/r5, -3*y**2/r5 + 1/r3],
        [1, 0, 0, 0],
        [0, 1, 0, 0]
    ])

def initial_approximation(zn, h):
    r2 = zn[2]**2 + zn[3]**2
    r3 = r2 ** (3/2)
    r5 = r2 ** (5/2)

    A = np.eye(4) - (h/2) * Df(zn)
    b = f(zn)

    k3 = (zn[2] + (h/2) * (-3*zn[2]*zn[3]/r5) * zn[3]) / (1 - (h/2) * (-3*zn[2]**2/r5 + 1/r3))
    k4 = (zn[3] + (h/2) * (-3*zn[2]*zn[3]/r5) * zn[2]) / (1 - (h/2) * (-3*zn[3]**2/r5 + 1/r3))

    k1 = zn[2]/r3 + (h/2) * k3
    k2 = zn[3]/r3 + (h/2) * k4

    return np.array([k1, k2, k3, k4])

# Example usage
zn = np.array([0, 0.5, 2, 0])  # Initial values
h = 0.01  # Time step

k_initial = initial_approximation(zn, h)
print("Initial approximation for k:")
print(k_initial)

Initial approximation for k:
[0.25998752 0.         1.99750312 0.        ]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        x * r**(-3/2),
        y * r**(-3/2),
        u,
        v
    ])

def Df(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        [0, 0, -3*x**2/r**5 + 1/r**(3), -3*x*y / r**(5)],
        [0, 0, -3*x*y / r**(5), -3*y**2/r**5 + 1/r**(3)],
        [1, 0, 0, 0],
        [0, 1, 0, 0]
    ])

def implicit_midpoint(z0, h, n_steps):
    z = np.zeros((n_steps+1, 4))
    z[0] = z0
    
    for i in range(n_steps):
        k = f(z[i]) + 0.5 * h * np.dot(Df(z[i]), f(z[i]))
        for _ in range(10):
            k = f(z[i] + 0.5 * h * k) + 0.5 * h * np.dot(Df(z[i] + 0.5 * h * k), k)
        z[i+1] = z[i] + h * k
    
    return z

# Parameters
u0, v0, x0, y0 = 0, 0.5, 2, 0
z0 = np.array([u0, v0, x0, y0])
T = 2 * np.pi * (4/3)**(3/2)  # Exact period of revolution
n_revs = 10

# Integrate with different time steps
steps_per_period = [100, 1000, 10000]

for spp in steps_per_period:
    n_steps = n_revs * spp
    h = T / spp
    z = implicit_midpoint(z0, h, n_steps)
    
    # Plot x and y components
    plt.figure()
    plt.plot(z[:, 2], z[:, 3])
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Trajectory (steps per period = {spp})')
    plt.axis('equal')
    
    # Plot Hamiltonian versus time
    H = 0.5 * (z[:, 0]**2 + z[:, 1]**2) - 1 / np.sqrt(z[:, 2]**2 + z[:, 3]**2)
    t = np.linspace(0, n_revs*T, n_steps+1)
    plt.figure()
    plt.plot(t, H)
    plt.xlabel('Time')
    plt.ylabel('Hamiltonian')
    plt.title(f'Hamiltonian vs Time (steps per period = {spp})')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        x / r**3,
        y / r**3,
        u,
        v
    ])

def Df(z):
    u, v, x, y = z
    r = np.sqrt(x**2 + y**2)
    return np.array([
        [0, 0, -3*x**2/r**5 + 1/r**(3), -3*x*y / r**(5)],
        [0, 0, -3*x*y / r**(5), -3*y**2/r**5 + 1/r**(3)],
        [1, 0, 0, 0],
        [0, 1, 0, 0]
    ])

def implicit_midpoint(z0, h, n_steps):
    z = np.zeros((n_steps+1, 4))
    z[0] = z0
    
    for i in range(n_steps):
        k = f(z[i]) + 0.5 * h * np.dot(Df(z[i]), f(z[i]))
        for _ in range(10):
            k = f(z[i] + 0.5 * h * k) + 0.5 * h * np.dot(Df(z[i] + 0.5 * h * k), k)
        z[i+1] = z[i] + h * k
    
    return z

# Parameters
u0, v0, x0, y0 = 0, 0.5, 2, 0
z0 = np.array([u0, v0, x0, y0])
T = 2 * np.pi * (4/3)**(3/2)  # Exact period of revolution
n_revs = 10

# Integrate with different time steps
steps_per_period = [100, 1000, 10000]

for spp in steps_per_period:
    n_steps = n_revs * spp
    h = T / spp
    z = implicit_midpoint(z0, h, n_steps)
    
    # Plot x and y components
    plt.figure()
    plt.plot(z[:, 2], z[:, 3])
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Trajectory (steps per period = {spp})')
    plt.axis('equal')
    
    # Plot Hamiltonian versus time
    H = 0.5 * (z[:, 0]**2 + z[:, 1]**2) - 1 / np.sqrt(z[:, 2]**2 + z[:, 3]**2)
    t = np.linspace(0, n_revs*T, n_steps+1)
    plt.figure()
    plt.plot(t, H)
    plt.xlabel('Time')
    plt.ylabel('Hamiltonian')
    plt.title(f'Hamiltonian vs Time (steps per period = {spp})')

plt.show()